In [1]:
import torch
from torch import nn as nn
import torch.optim as optim
import time
import numpy as np
import random
import torch.nn.functional as F

In [2]:
load_val = torch.load('valset_batchSize=64.pt')


In [4]:
print(load_val[0].shape)

torch.Size([13434, 16, 64, 14])


In [4]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(device)

cpu


In [6]:
def prepare_data(batch_size, t_h): #假设 t_h = 8
    inputs = torch.randn(batch_size, t_h, 14) # len(features) = 14
    return inputs    # batch_size, seq_length, features

In [128]:
ttttt = prepare_data(20, 8)
input_batch = ttttt[ 0: 5,:, :]
print(ttttt.shape)
print(input_batch.shape)

torch.Size([20, 8, 14])
torch.Size([5, 8, 14])


In [70]:
yyy = torch.rand(6,20,128)
zzz = yyy[0].view(1,20,-1)
print(zzz.shape)

torch.Size([1, 20, 128])


In [79]:
ttt = torch.empty(6,20,6)
print(ttt[0].shape)
cla = torch.zeros(6,6)
print(cla[0].shape)
cla[0].view(1,-1).shape

torch.Size([20, 6])
torch.Size([6])


torch.Size([1, 6])

In [81]:
def one_hot_coding():
    m_class = torch.zeros(6,6)
    for i in range(0,6):
        m_class[i][i] = 1
    one_hot = torch.empty(6,20,6)
    for i in range(6):
        temp = m_class[i].view(1,-1)
        for j in range(0, 19):
            
            temp = torch.cat((temp,m_class[i].view(1,-1)))
        one_hot[i] = temp
    return one_hot

one_hot = one_hot_coding()
print(one_hot.shape)

torch.Size([6, 20, 6])


## Load Data

In [10]:
popoopo= nn.LSTM(64, hidden_size = 128,
                            num_layers = 1, batch_first=True)

In [130]:
class lstm_encoder(nn.Module):
    ''' Encodes time-series sequence '''

    def __init__(self, input_size, hidden_size=128, num_layers = 1):
        
        '''
        : param input_size:     the number of features in the input X
        : param hidden_size:    the number of features in the hidden state h
        : param num_layers:     number of recurrent layers (i.e., 2 means there are
        :                       2 stacked LSTMs)
        '''
        
        super(lstm_encoder, self).__init__()
        self.input_size = input_size
        self.embedding = nn.Linear(input_size, 64)
        self.hidden_size = 128
        self.num_layers = 1

        # define LSTM layer
        self.lstm = nn.LSTM(64, hidden_size = 128,                        # input 是 (batch_size, sequence_size, input_size)
                            num_layers = 1, batch_first=True)  # input为序列中单一元素的维度，因为是一个一个输进去的

        #define activation:
        self.leaky_relu = torch.nn.LeakyReLU(0.1)
        
    def forward(self, x_input):
        
        '''
        : param x_input:               input of shape (seq_len, # in batch, input_size)
        : return lstm_out, hidden:     lstm_out gives all the hidden states in the sequence;
        :                              hidden gives the hidden state and cell state for the last
        :                              element in the sequence 
        '''
        embedded = self.embedding(x_input)
        embedded = self.leaky_relu(embedded)
        lstm_out, self.hidden = self.lstm(embedded)
        
        
        return lstm_out, self.hidden     
    
    def init_hidden(self, batch_size):
        
        '''
        initialize hidden state
        : param batch_size:    x_input.shape[1]
        : return:              zeroed hidden state and cell state 
        '''
        # (self.n_layers, batch_size, self.hidden_dim)
        return (torch.zeros(self.num_layers, batch_size, self.hidden_size),
                torch.zeros(self.num_layers, batch_size, self.hidden_size))


In [118]:
torch.manual_seed(1729)
inputs = prepare_data(20, 8)
input_sizes = 14
encoder = lstm_encoder(input_sizes)

In [119]:
output, hidden = encoder(inputs)
print(output.shape)
print(hidden[0].shape)
print(hidden[1].shape)

torch.Size([20, 8, 128])
torch.Size([1, 20, 128])
torch.Size([1, 20, 128])


从Encoder到decoder的维度拼接：

In [120]:
h_d0 = torch.cat((hidden[0], one_hot[0].view(1,20,-1)),2)
c_d0 = torch.cat((hidden[1], one_hot[0].view(1,20,-1)),2)
cat_hidden_states = (h_d0, c_d0)


In [121]:
class lstm_decoder(nn.Module):
    ''' Decodes hidden state output by encoder '''
    
    def __init__(self, input_size, output_size, hidden_size=128, num_layers = 1):

        '''
        : param input_size:     the number of features in the input X
        : param hidden_size:    the number of features in the hidden state h, unit number = 128
        : param num_layers:     number of recurrent layers (i.e., 2 means there are
        :                       2 stacked LSTMs)
        '''
        
        super(lstm_decoder, self).__init__()
        self.input_size = input_size
        self.hidden_size = 134
        self.num_layers = 1
        #self.embedding = nn.Linear(input_size, 64)
        self.lstm = nn.LSTM(128, hidden_size = self.hidden_size,
                            num_layers = self.num_layers, batch_first=True)
        self.linear = nn.Linear(self.hidden_size, output_size)           

    def forward(self, encoder_input, encoder_hidden_states_cat):
        
        '''        
        : param x_input:                    should be 2D (batch_size, input_size)
        : param encoder_hidden_states:      hidden states
        : return output, hidden:            output gives all the hidden states in the sequence;
        :                                   hidden gives the hidden state and cell state for the last
        :                                   element in the sequence 
 
        '''
        lstm_out, self.hidden = self.lstm(encoder_input, encoder_hidden_states_cat)   
        output = self.linear(lstm_out)      
        
        return output, self.hidden

In [134]:
class lstm_seq2seq(nn.Module):
    ''' train LSTM encoder-decoder and make predictions '''
    
    def __init__(self, enc_input_size, hidden_size):

        super(lstm_seq2seq, self).__init__()

        self.enc_input_size = enc_input_size
        self.hidden_size = hidden_size
        self.output_size = 2

        self.encoder = lstm_encoder(self.enc_input_size)
        self.decoder = lstm_decoder(input_size=128, output_size=2)


    def train_model(self, input_tensor, target_tensor,  hot, n_epochs, target_len, batch_size, training_prediction = 'recursive', lr = 0.001):
        
        '''
        train lstm encoder-decoder
        
        : param input_tensor:              input data with shape (seq_len, # in batch, number features); PyTorch tensor    
        : param target_tensor:             target data with shape (seq_len, # in batch, number features); PyTorch tensor
        : param n_epochs:                  number of epochs 
        : param target_len:                number of values to predict 
        : param batch_size:                number of samples per gradient update
        : param training_prediction:       type of prediction to make during training ('recursive', 'teacher_forcing', or
        :                                  'mixed_teacher_forcing'); default is 'recursive'
        : param teacher_forcing_ratio:     float [0, 1) indicating how much teacher forcing to use when
        :                                  training_prediction = 'teacher_forcing.' For each batch in training, we generate a random
        :                                  number. If the random number is less than teacher_forcing_ratio, we use teacher forcing.
        :                                  Otherwise, we predict recursively. If teacher_forcing_ratio = 1, we train only using
        :                                  teacher forcing.
        : param learning_rate:             float >= 0; learning rate
        : param dynamic_tf:                use dynamic teacher forcing (True/False); dynamic teacher forcing
        :                                  reduces the amount of teacher forcing for each epoch
        : return losses:                   array of loss function for each epoch
        '''
        
        # initialize array of losses 
        
        loss_fn = nn.NLLLoss()
        optimizer = torch.optim.Adam(self.parameters(), lr=0.001)
        
        # calculate number of batch iterations
        n_batches = int(input_tensor.shape[0] / batch_size)

        for it in range(n_epochs):
            start = time.time()
            
            print(f"Epoch {it+1}\n-------------------------------")
            batch_loss = 0.
            num = 0
            for b in range(n_batches):
                # select data 
                input_batch = input_tensor[b: b + batch_size, :, :]
                target_batch = target_tensor[b: b + batch_size, :, :]

                # outputs tensor
                outputs = torch.zeros(batch_size, target_len, self.output_size)

                # initialize hidden state
                encoder_hidden = self.encoder.init_hidden(batch_size)

                # zero the gradient
                optimizer.zero_grad()

                # encoder outputs
                encoder_output, encoder_hidden = self.encoder(input_batch)

                # use concat state as decoder's hidden states 
                decoder_input = encoder_output
                for j in range(0,6):
                    h_d = torch.cat((encoder_hidden[0], one_hot[j].view(1,20,-1)),2)
                    c_d = torch.cat((encoder_hidden[1], one_hot[j].view(1,20,-1)),2)
                    cat_hidden_states = (h_d0, c_d0)
                    decoder_hidden = cat_hidden_states
                    
                    #decoder outputs 
                    pred, decoder_output_hidden = self.decoder(decoder_input, decoder_hidden)
                    
                    #computing loss
                    loss = loss_fn(outputs.transpose(1,2), target_batch)
                    batch_loss += loss.item()
                    
                # backpropagation
                loss.backward()
                optimizer.step()
            batch_loss /= n_batches
            losses = batch_loss
            
            # calculate iteration
            
            
        
        return losses 
                    
                    


In [122]:
decoder = lstm_decoder(input_size=128, output_size=2)
decoder_output, decoder_hidden = decoder(output, cat_hidden_states)
print(decoder_output.shape)

torch.Size([20, 8, 2])


# training

In [123]:
# training


# Evaluate：

In [ ]:
def evaluate():
    